In [1]:
# Librerias utilizadas

import requests
from PIL import Image, ImageOps,ImageFilter
import pytesseract
import pandas as pd
from os.path import join, exists
import cv2
from matplotlib import pyplot as plt
import re

C:\Users\XAVIER\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import lotto

INSERT INTO SORTEOS ( ID, FECHA ) VALUES ( :id, :fecha )
INSERT INTO PREMIADOS ( ID_SORTEO, ORDEN, PREMIADO ) VALUES ( :id_sorteo, :orden, :premiado )
INSERT INTO PROBABILIDADES ( POSICION, DIGITO, PROBABILIDAD, RANKING ) VALUES ( :posicion, :digito, :probabilidad, :ranking )


In [ ]:
lotto.inicializar()

In [ ]:
lotto.transaccionar(lotto.poblar_probabilidades, {})

In [3]:
# Paths relativos
PIC_FOLDER = 'lotto_pics'
URL_BASE = "https://ventas-api.loteria.com.ec/uploads/boletines/T{0}.jpg"

In [6]:
# Ultimo Sorteo
ULTIMO_SORTEO = 23072

In [10]:
# Extrae ultimas Imagenes
sorteo = lotto.find_ultimo_sorteo()[0] + 1
while True:
    path_archivo = join(PIC_FOLDER,"{0}.jpg".format(sorteo))
    if exists(path_archivo):
        print('Boletin {0} ya existe'.format(sorteo))
        sorteo += 1
        continue
    respuesta = requests.get(URL_BASE.format(sorteo), verify = False)
    if respuesta.status_code != 200:        
        respuesta.close()
        print('Boletin {0} no esta disponible'.format(sorteo))
        break
    archivo = open( path_archivo, "wb" )
    archivo.write(respuesta.content)
    archivo.close()
    print('Boletin {0} extraido'.format(sorteo))
    sorteo += 1
print('Secuencia finalizada')

Boletin 23054 ya existe
Boletin 23055 ya existe
Boletin 23056 ya existe
Boletin 23057 ya existe
Boletin 23058 ya existe
Boletin 23059 ya existe
Boletin 23060 ya existe
Boletin 23061 ya existe
Boletin 23062 ya existe
Boletin 23063 ya existe
Boletin 23064 ya existe
Boletin 23065 ya existe
Boletin 23066 ya existe
Boletin 23067 ya existe
Boletin 23068 ya existe
Boletin 23069 ya existe
Boletin 23070 ya existe
Boletin 23071 ya existe
Boletin 23072 ya existe


C:\Users\XAVIER\AppData\Roaming\Python\Python310\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ventas-api.loteria.com.ec'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Boletin 23073 no esta disponible
Secuencia finalizada


In [12]:
def  extraer_numeros_loteria(numero_sorteo, path_archivo, umbral = 242, mostrar_imagen=False, solo_numeros_loteria = True):
    imagen = cv2.imread(path_archivo, cv2.IMREAD_GRAYSCALE)
    imagen_umbralizada = imagen
    ret, imagen_umbralizada = cv2.threshold(imagen, umbral, 255, cv2.THRESH_BINARY)    
    tamano = imagen_umbralizada.shape[:2]
    imagen_umbralizada = imagen_umbralizada[ : int(tamano[0] * 0.55) ,  :  ]
    
    hallazgos = pytesseract.image_to_string( imagen_umbralizada ).split()
    expresion_num_loteria = re.compile(r'^\d{6}$')
    if solo_numeros_loteria:
        hallazgos = [ x for x in hallazgos if expresion_num_loteria.match(x) ]
    hallazgos1 = []
    sorteo = {'id': numero_sorteo, 'fecha': ''}
    lotto.transaccionar(lotto.crear_sorteo, sorteo)
    orden = 1
    for h in hallazgos:
        if h not in hallazgos1:
            premiado = {'id_sorteo': numero_sorteo, 'orden':orden, 'premiado': h}
            lotto.transaccionar(lotto.crear_premiado, premiado)
            hallazgos1.append(h)
            orden += 1
    if mostrar_imagen:
        print(imagen_umbralizada.shape)
        plt.figure(figsize=(10,10))
        plt.imshow(cv2.cvtColor(imagen_umbralizada, cv2.COLOR_GRAY2BGR))
        plt.show()
        plt.close()
    
    return hallazgos1


In [13]:
# Persiste los sorteos desde imagenes

sorteo = lotto.find_ultimo_sorteo()[0] + 1
umbrales = [242,128,64]
while True:
    path_archivo = join(PIC_FOLDER,"{0}.jpg".format(sorteo))
    if not exists(path_archivo):
        print("info para sorteo {0} no disponible".format(sorteo))
        break
    lotto.find_ultimo_sorteo()
    numeros_encontrados = []
    for umbral in umbrales:
        numeros_encontrados1 = extraer_numeros_loteria ( sorteo, path_archivo , umbral= 242 )
        if len(numeros_encontrados1) > len(numeros_encontrados):
            numeros_encontrados = numeros_encontrados1
    linea = "{0},{1}".format(sorteo,','.join(numeros_encontrados))
    # archivo = open(path_data,'at')
    # print(linea, file=archivo)
    print(linea)
    # archivo.close()
    sorteo += 1
print("Secuencia terminada")

23054,268061,661235,133603,847714,999074,111268,164034,449917,946394
23055,479494,223728,262276,242367,534717,027744,159994
23056,468643,999349,960272,330530,598303,476183,429906,935555,540096,228861,683755
23057,257023,983926,197492,885619,471383,813185,483045,593018,782294,038825,934393
23058,
23059,688732,214921,559971,028439,230816,381452,142006,335930,591044,845591,546657
23060,229858,787966,147692,178952,195426,778990,221154,307550,596918,152971,002531
23061,138839,498149,572565,899941,193043,433871,395743,851171,352397,286384,283880
23062,699937,453448,201991,547754,624832,550237,717800,307239,548260
23063,488244,068128,517757,022430,790347,772345,068654,169736,812928
23064,183195,426845,472025,959985,565477,846075,763728,886446,165677,885170
23065,164197,634848,046728,066827,571033,750659,964958,211955,569951,795163
23066,010884,877161,092694,458794,940394,352713,654212,156955,144343,565597,959367
23067,616806,604203,208350,693485,133206,561719,579261,721501,869682,918906
23068

In [14]:
lotto.calcular_probabilidades()

In [15]:
lotto.mostrar_probabilidad('000000')

['warning', 'warning', 'danger', 'success', 'success', 'success']

In [ ]:
archivo.close()

In [ ]:
# Se cara sorteos como dataframe
from nltk.probability import FreqDist

path_data = "salida.txt"
sorteos = []
with open(path_data, 'rt') as archivo:    
    for linea in archivo:
        arreglo_linea = linea[:-1].split(',')
        tamano_arreglo = len(arreglo_linea) - 1
        if tamano_arreglo < 2:
            continue
        favorecidos = arreglo_linea[1:]
        print(favorecidos)
        digitos = [ [x[pos] for x in favorecidos ] for pos in range(0,6) ]
        distribuciones = [ str(dict(FreqDist(pos))) for pos in digitos ]
        registro = {'SORTEO': int(arreglo_linea[0]) , 'FAVORECIDOS': ','.join(favorecidos), 'CONTEO FAVORECIDOS': tamano_arreglo }
        for dpos in range(0,6):
            registro['D{0}'.format(dpos+1)] = distribuciones[dpos]
        sorteos.append(registro)

sorteos = pd.DataFrame(sorteos)
sorteos


In [ ]:
# Conteo de Sorteos por conteo favorecidos\
sorteos[['SORTEO','CONTEO FAVORECIDOS']].groupby(['CONTEO FAVORECIDOS']).count()

In [ ]:
sorteos.loc[sorteos['CONTEO FAVORECIDOS'] == 17]

In [ ]:
from nltk.probability import FreqDist

linea = '22895,906085,124822,981826,998854,347723,777305,553624,774288,903696,127377,375818,936178,787748,156995,472867,275541'
arreglo_linea = linea.split(',')
tamano_arreglo = len(arreglo_linea) - 1
favorecidos = arreglo_linea[1:]
digitos = [ [x[pos] for x in favorecidos ] for pos in range(0,6) ]
distribuciones = [ str(dict(FreqDist(pos))) for pos in digitos ]
distribuciones

In [ ]:
#import nltk
#nltk.download('stopwords')

from pyresparser import ResumeParser

data = ResumeParser('C:\\Users\\XAVIER\\Documents\\RESUME-Marcelo Merizalde.pdf').get_extracted_data()

In [ ]:
# Persistiendo 
chr(65)